In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import pickle
import json
%matplotlib inline


In [2]:
df1 = pd.read_csv('data/rent_apts.csv')
df1.head()

,Agency,Neighborhood,Price,link,sq_mtrs,Bedrooms,Bathrooms
0,Buy Rent Shelters,"General Mathenge, Westlands","KSh 155,000",/listings/4-bedroom-apartment-for-rent-general...,4.0,4.0,4.0
1,Kenya Classic Homes,"Kilimani, Dagoretti North","KSh 100,000",/listings/3-bedroom-apartment-for-rent-kiliman...,300.0,3.0,4.0
2,Absolute Estate Agents,"Hatheru Rd,, Lavington, Dagoretti North","KSh 75,000",/listings/3-bedroom-apartment-for-rent-lavingt...,3.0,3.0,5.0
3,A1 Properties Limited,"Kilimani, Dagoretti North","KSh 135,000",/listings/3-bedroom-apartment-for-rent-kiliman...,227.0,3.0,4.0
4,Pmc Estates Limited,"Imara Daima, Embakasi","KSh 50,000",/listings/3-bedroom-apartment-for-rent-imara-d...,3.0,3.0,NaN


In [3]:
df1.describe()

,sq_mtrs,Bedrooms,Bathrooms
count,1846.000000,1845.000000,1557.000000
mean,327.226977,2.577778,2.595376
std,2048.784581,0.819399,1.000349
min,0.000000,0.000000,1.000000
25%,2.000000,2.000000,2.000000
50%,3.000000,3.000000,2.000000
75%,4.000000,3.000000,3.000000
max,40470.000000,6.000000,6.000000


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1848 entries, 0 to 1847
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Agency        1848 non-null   object 
 1   Neighborhood  1848 non-null   object 
 2   Price         1848 non-null   object 
 3   link          1848 non-null   object 
 4   sq_mtrs       1846 non-null   float64
 5   Bedrooms      1845 non-null   float64
 6   Bathrooms     1557 non-null   float64
dtypes: float64(3), object(4)
memory usage: 101.2+ KB


In [15]:
df1.Agency.unique().count

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [11]:
df1.groupby('Neighborhood')['Neighborhood'].agg('count')

Neighborhood
, Dennis Pritt, Kilimani, Dagoretti North            1
01, Marula Lane, Kilimani, Dagoretti North           1
04, kilimani, Kilimani, Dagoretti North              1
1 Kilimani, Kilimani, Dagoretti North                1
1 Ojijo, Parklands, Westlands                        1
                                                    ..
walk to valley arcade, Lavington, Dagoretti North    1
wambugu road, 1st parkland, Parklands, Westlands     1
westlands Avenue , Westlands Area, Westlands         1
westlands, Westlands Area, Westlands                 2
westlands, Westlands, Riverside, Westlands           1
Name: Neighborhood, Length: 647, dtype: int64

In [16]:
df1['Agency'].value_counts()

Agency
Prudential Green Ville            98
Wealthlink Realtors Ltd           67
Buy Rent Shelters                 59
Pam Golding Properties            51
Mambo Properties Enterprise       51
                                  ..
Archstone Realtors                 1
Hold Kenya Investments Limited     1
Royalgen Estates                   1
Sunflag Kenya                      1
Rainbow Properties Limited         1
Name: count, Length: 194, dtype: int64

In [23]:
df1[(df1['Agency'].value_counts()  <= 1)]

C:\Users\user\AppData\Local\Temp\ipykernel_13676\196160112.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1[(df1['Agency'].value_counts()  <= 1)]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [21]:
df1['Agency'].unique()

array(['Buy Rent Shelters', 'Kenya Classic Homes',
       'Absolute Estate Agents', 'A1 Properties Limited',
       'Pmc Estates Limited', 'Lizmor Properties',
       'Mambo Properties Enterprise', 'Wealthlink Realtors Ltd',
       'Naftal Properties', 'Dakima Home International',
       'Opus Properties', 'Realty Plus', 'Pam Golding Properties',
       'Yellow Market', 'Hyra Real Estate Ltd',
       'Dunhill Consulting Limited', 'Green Knight Properties',
       'Kitheka Musyoka', 'Real Management Services 2002Limited',
       'Estate Masters Limited', 'Homekenya24',
       'Habitat Realtors International Ltd', 'Royani Properties Limited',
       'Unique Casles Management Ltd', 'Wainaina Real Estate Agents',
       'Havilaplus Estates Ltd', 'Ngong Hills Agency',
       'Chelezo Residency', 'Origin Homes', 'Cfs Homes',
       'Canaan Properties', 'Kimachas Enterprises', 'The Epic Properties',
       'Westside Real Estate', 'Tysons Limited', 'Crystal Valuers Ltd',
       'Swiveltrend Lt

In [29]:
df1['Agency'].describe()


count                       1848
unique                       194
top       Prudential Green Ville
freq                          98
Name: Agency, dtype: object

In [25]:
df1.isnull().sum()

Agency            0
Neighborhood      0
Price             0
link              0
sq_mtrs           2
Bedrooms          3
Bathrooms       291
dtype: int64

In [30]:
df1.shape


(1848, 7)

In [32]:
df1['Agency'].info

<bound method Series.info of 0             Buy Rent Shelters
1           Kenya Classic Homes
2        Absolute Estate Agents
3         A1 Properties Limited
4           Pmc Estates Limited
                 ...           
1843         Avic International
1844              Dmus Realtors
1845      A1 Properties Limited
1846    Geoscape Global Limited
1847        Pmc Estates Limited
Name: Agency, Length: 1848, dtype: object>

In [33]:
df1


,Agency,Neighborhood,Price,link,sq_mtrs,Bedrooms,Bathrooms
0,Buy Rent Shelters,"General Mathenge, Westlands","KSh 155,000",/listings/4-bedroom-apartment-for-rent-general...,4.0,4.0,4.0
1,Kenya Classic Homes,"Kilimani, Dagoretti North","KSh 100,000",/listings/3-bedroom-apartment-for-rent-kiliman...,300.0,3.0,4.0
2,Absolute Estate Agents,"Hatheru Rd,, Lavington, Dagoretti North","KSh 75,000",/listings/3-bedroom-apartment-for-rent-lavingt...,3.0,3.0,5.0
3,A1 Properties Limited,"Kilimani, Dagoretti North","KSh 135,000",/listings/3-bedroom-apartment-for-rent-kiliman...,227.0,3.0,4.0
4,Pmc Estates Limited,"Imara Daima, Embakasi","KSh 50,000",/listings/3-bedroom-apartment-for-rent-imara-d...,3.0,3.0,NaN
...,...,...,...,...,...,...,...
1843,Avic International,"Laikipia Road, Kileleshwa, Dagoretti North","KSh 180,000",/listings/3-bedroom-apartment-for-rent-kileles...,3.0,3.0,NaN
1844,Dmus Realtors,"Peponi road, Spring Valley, Westlands","KSh 180,000",/listings/2-bedroom-apartment-for-rent-spring-...,2.0,2.0,2.0
1845,A1 Properties Limited,"Kileleshwa, Dagoretti North","KSh 180,000",/listings/furnished-4-bedroom-apartment-for-re...,4.0,4.0,5.0
1846,Geoscape Global Limited,"Westlands Area, Westlands","KSh 180,000",/listings/4-bedroom-apartment-for-rent-westlan...,4.0,4.0,4.0


In [34]:
df2 = df1.drop(['link'],axis='columns')
df2.shape

(1848, 6)

In [35]:
df2.head()

,Agency,Neighborhood,Price,sq_mtrs,Bedrooms,Bathrooms
0,Buy Rent Shelters,"General Mathenge, Westlands","KSh 155,000",4.0,4.0,4.0
1,Kenya Classic Homes,"Kilimani, Dagoretti North","KSh 100,000",300.0,3.0,4.0
2,Absolute Estate Agents,"Hatheru Rd,, Lavington, Dagoretti North","KSh 75,000",3.0,3.0,5.0
3,A1 Properties Limited,"Kilimani, Dagoretti North","KSh 135,000",227.0,3.0,4.0
4,Pmc Estates Limited,"Imara Daima, Embakasi","KSh 50,000",3.0,3.0,NaN


In [36]:
df2.isnull().sum()

Agency            0
Neighborhood      0
Price             0
sq_mtrs           2
Bedrooms          3
Bathrooms       291
dtype: int64

In [38]:
df2.Bathrooms.isnull().value_counts

<bound method IndexOpsMixin.value_counts of 0       False
1       False
2       False
3       False
4        True
        ...  
1843     True
1844    False
1845    False
1846    False
1847     True
Name: Bathrooms, Length: 1848, dtype: bool>